 ### Домашнее задание по теме «Spark SQL»

Задача 1. Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [650]:
import findspark
findspark.init(r'C:\SPARK\spark-3.4.0-bin-hadoop3')
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [651]:
import pyspark
from pyspark.sql import SparkSession

In [652]:
spark = SparkSession.builder.getOrCreate()

In [653]:
df = spark.read.csv(r'C:\Users\user\Documents\IT\NETOLOGY\APACHE SPARK\owid-covid-data.csv')

In [654]:
type(df)

pyspark.sql.dataframe.DataFrame

In [655]:
df = spark.read.option('inferSchema', 'true')\
.option('header', 'true').csv(r'C:\Users\user\Documents\IT\NETOLOGY\APACHE SPARK\owid-covid-data.csv')

In [656]:
from pyspark.sql.functions import col
from pyspark.sql.functions import dayofmonth
from pyspark.sql import functions as F
from pyspark.sql.functions import sum
from pyspark.sql.functions import round 
from pyspark.sql.functions import date_format
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
from pyspark.sql import Window as W
from pyspark.sql.functions import row_number
from pyspark.sql.functions import max
from pyspark.sql.functions import lag

In [ ]:
# список не стран, который мы исключим из выборки


In [657]:
excluded = ['Europe', 'European Union', 'South America', 'North America', 'World', 'Asia', 'Africa']

In [658]:
# сначала просто отфильтруем данные по 31 марта 2021, исключим из списка не страны, колонки total_cases и 
# population с Null значениями
df_1 = df.select('iso_code', 'location', 'date', 'total_cases', 'population')\
.where((df['date']=='2021-03-31') & (~df.location.isin(excluded)) & (F.col('total_cases').isNotNull())\
& (F.col('population').isNotNull()))

In [659]:
# далее оставляем нужные нам колонки iso_code и location и считаем процент переболевших как total_cases*100/population 
# и сортируем по убыванию
df_1 = df_1.select('iso_code', 'location', (round(F.col('total_cases')*100/F.col('population'),2)).alias('recovered'))\
.orderBy(F.col('recovered').desc()).show(15)

+--------+-------------+---------+
|iso_code|     location|recovered|
+--------+-------------+---------+
|     AND|      Andorra|    15.54|
|     MNE|   Montenegro|    14.52|
|     CZE|      Czechia|    14.31|
|     SMR|   San Marino|    13.94|
|     SVN|     Slovenia|    10.37|
|     LUX|   Luxembourg|     9.85|
|     ISR|       Israel|     9.63|
|     USA|United States|      9.2|
|     SRB|       Serbia|     8.83|
|     BHR|      Bahrain|     8.49|
|     PAN|       Panama|     8.23|
|     PRT|     Portugal|     8.06|
|     EST|      Estonia|     8.02|
|     SWE|       Sweden|     7.97|
|     LTU|    Lithuania|     7.94|
+--------+-------------+---------+
only showing top 15 rows



Задача 2. Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [660]:
# В колонке location помимо стран у нас содержатся также регионы и континенты
# Составим список чтобы потом в фильтре их исключить из выборки
excluded = ['Europe', 'European Union', 'South America', 'North America', 'World', 'Asia', 'Africa']

In [ ]:
# 1 способ решения с помощью оконных функций

In [661]:
# выделим год месяц и номер недели и искючим из выборки не страны
df_3 = df.select('date', date_format(col('date'), 'M').alias('Month'),date_format(col('date'), 'W')\
.alias('week_num'), date_format(col('date'), 'Y').alias('Year'),date_format(col('date'), 'u')\
.alias('week_day_num'), date_format(col('date'), 'E').alias('week_day_abb'), 'location', 'new_cases')\
.where(~df.location.isin(excluded))

In [662]:
df_3.show(5)

+----------+-----+--------+----+------------+------------+-----------+---------+
|      date|Month|week_num|Year|week_day_num|week_day_abb|   location|new_cases|
+----------+-----+--------+----+------------+------------+-----------+---------+
|2020-02-24|    2|       5|2020|           1|         Mon|Afghanistan|      1.0|
|2020-02-25|    2|       5|2020|           2|         Tue|Afghanistan|      0.0|
|2020-02-26|    2|       5|2020|           3|         Wed|Afghanistan|      0.0|
|2020-02-27|    2|       5|2020|           4|         Thu|Afghanistan|      0.0|
|2020-02-28|    2|       5|2020|           5|         Fri|Afghanistan|      0.0|
+----------+-----+--------+----+------------+------------+-----------+---------+
only showing top 5 rows



In [663]:
# отфильтруем данные по последней неделе марта 2021 года
df_3 = df_3.select('date','week_num','week_day_num', 'week_day_abb', 'location', 'new_cases')\
.where((df_3['Year']==2021) & (df_3['Month']==3) & (df_3['week_num']==5))

In [664]:
df_3.show(5)

+----------+--------+------------+------------+-----------+---------+
|      date|week_num|week_day_num|week_day_abb|   location|new_cases|
+----------+--------+------------+------------+-----------+---------+
|2021-03-28|       5|           7|         Sun|Afghanistan|      4.0|
|2021-03-29|       5|           1|         Mon|Afghanistan|     28.0|
|2021-03-30|       5|           2|         Tue|Afghanistan|     62.0|
|2021-03-31|       5|           3|         Wed|Afghanistan|     70.0|
|2021-03-28|       5|           7|         Sun|    Albania|    493.0|
+----------+--------+------------+------------+-----------+---------+
only showing top 5 rows



In [665]:
# c помощью оконной функции отсортируем данные по странам и с помощью row_number присвоим наибольшему количеству
# new_cases номер 1
df_3 = df_3.select('*', F.row_number().over(W.partitionBy('location').orderBy(col('new_cases').desc())).alias('row_num'))

In [667]:
df_3.show(5)

+----------+--------+------------+------------+-----------+---------+-------+
|      date|week_num|week_day_num|week_day_abb|   location|new_cases|row_num|
+----------+--------+------------+------------+-----------+---------+-------+
|2021-03-31|       5|           3|         Wed|Afghanistan|     70.0|      1|
|2021-03-30|       5|           2|         Tue|Afghanistan|     62.0|      2|
|2021-03-29|       5|           1|         Mon|Afghanistan|     28.0|      3|
|2021-03-28|       5|           7|         Sun|Afghanistan|      4.0|      4|
|2021-03-28|       5|           7|         Sun|    Albania|    493.0|      1|
+----------+--------+------------+------------+-----------+---------+-------+
only showing top 5 rows



In [668]:
# отфильтруем наши данные по row_number 1 и расположим количество new_cases в порядке убывания и выведем 15 строк
df_3.select('date', 'location', 'new_cases',).where(df_3['row_num']==1).orderBy(F.desc('new_cases')).show(15)

+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-03-31|       Brazil|  90638.0|
|2021-03-31|        India|  72330.0|
|2021-03-29|United States|  69429.0|
|2021-03-31|       France|  59054.0|
|2021-03-31|       Turkey|  39302.0|
|2021-03-31|       Poland|  32891.0|
|2021-03-31|      Germany|  25014.0|
|2021-03-31|        Italy|  23887.0|
|2021-03-30|       Sweden|  16427.0|
|2021-03-31|    Argentina|  16056.0|
|2021-03-31|         Peru|  15686.0|
|2021-03-29|        Spain|  15501.0|
|2021-03-28|      Ukraine|  12041.0|
|2021-03-31|         Iran|  10330.0|
|2021-03-29|  Philippines|  10002.0|
+----------+-------------+---------+
only showing top 15 rows



In [ ]:
# решим задачу 2 способом с помощью max и join

In [669]:
# отфильтруем наши данные по последней неделе марта и исключим не страны
df_4 = df.select('date', 'location', 'new_cases')\
.where((date_format(col('date'), 'Y')==2021) & (date_format(col('date'), 'M')==3)\
& (date_format(col('date'), 'W')==5) & (~df.location.isin(excluded)))

In [670]:
df_4.show()

+----------+-----------+---------+
|      date|   location|new_cases|
+----------+-----------+---------+
|2021-03-28|Afghanistan|      4.0|
|2021-03-29|Afghanistan|     28.0|
|2021-03-30|Afghanistan|     62.0|
|2021-03-31|Afghanistan|     70.0|
|2021-03-28|    Albania|    493.0|
|2021-03-29|    Albania|    285.0|
|2021-03-30|    Albania|    304.0|
|2021-03-31|    Albania|    434.0|
|2021-03-28|    Algeria|     86.0|
|2021-03-29|    Algeria|    110.0|
|2021-03-30|    Algeria|    115.0|
|2021-03-31|    Algeria|    131.0|
|2021-03-28|    Andorra|     41.0|
|2021-03-29|    Andorra|     38.0|
|2021-03-30|    Andorra|     56.0|
|2021-03-31|    Andorra|     66.0|
|2021-03-28|     Angola|     32.0|
|2021-03-29|     Angola|     69.0|
|2021-03-30|     Angola|     50.0|
|2021-03-31|     Angola|    129.0|
+----------+-----------+---------+
only showing top 20 rows



In [671]:
# найдем максимальное значение new_cases для страны и расположим их в порядке убывания
df_5 = df_4.groupBy('location').agg(max('new_cases').alias('max_cases')).orderBy(F.desc(max('new_cases')))

In [672]:
df_5.show()

+-------------+---------+
|     location|max_cases|
+-------------+---------+
|       Brazil|  90638.0|
|        India|  72330.0|
|United States|  69429.0|
|       France|  59054.0|
|       Turkey|  39302.0|
|       Poland|  32891.0|
|      Germany|  25014.0|
|        Italy|  23887.0|
|       Sweden|  16427.0|
|    Argentina|  16056.0|
|         Peru|  15686.0|
|        Spain|  15501.0|
|      Ukraine|  12041.0|
|         Iran|  10330.0|
|  Philippines|  10002.0|
|       Serbia|   9983.0|
|      Hungary|   9082.0|
|       Russia|   8979.0|
|       Canada|   8728.0|
|      Czechia|   8664.0|
+-------------+---------+
only showing top 20 rows



In [673]:
# с помощью join соединим два датафрейма по location и max_cases, уберем ненужные колонки и отсортируем количество 
# новых случаев в порядке убывания и выведем 15 строк. Проверяем первое и второе решение совпало.
df_4.join(df_5, (df_4["location"] == df_5["location"]) &( df_4["new_cases"] == df_5["max_cases"]))\
.drop(df_4['new_cases'], df_4['location']).orderBy(F.desc('max_cases')).show(15)

+----------+-------------+---------+
|      date|     location|max_cases|
+----------+-------------+---------+
|2021-03-31|       Brazil|  90638.0|
|2021-03-31|        India|  72330.0|
|2021-03-29|United States|  69429.0|
|2021-03-31|       France|  59054.0|
|2021-03-31|       Turkey|  39302.0|
|2021-03-31|       Poland|  32891.0|
|2021-03-31|      Germany|  25014.0|
|2021-03-31|        Italy|  23887.0|
|2021-03-30|       Sweden|  16427.0|
|2021-03-31|    Argentina|  16056.0|
|2021-03-31|         Peru|  15686.0|
|2021-03-29|        Spain|  15501.0|
|2021-03-28|      Ukraine|  12041.0|
|2021-03-31|         Iran|  10330.0|
|2021-03-29|  Philippines|  10002.0|
+----------+-------------+---------+
only showing top 15 rows



Задача 3. Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [675]:
# найдем даты которые соответствуют последней неделе марта 2021
res = df.select('date').where((date_format(col('date'), 'Y')==2021) & (date_format(col('date'), 'M')==3)\
& (date_format(col('date'), 'W')==5) & (F.col('location')=='Russia')).orderBy(F.col('date'))

In [676]:
res.show()

+----------+
|      date|
+----------+
|2021-03-28|
|2021-03-29|
|2021-03-30|
|2021-03-31|
+----------+



In [678]:
# чтобы в нашу последуюущую выборку включить количество новых случаев вчера для даты 2021-03-28 найдем дату на 1 день раньше
# и присовим это значение переменной start_date
start_date = res.select(F.date_sub(F.min('date'),1)).first()[0]
start_date

datetime.date(2021, 3, 27)

In [679]:
# введем переменную end_date и присвоим последнее значение даты в нашей выборке
end_date = res.select(F.max('date')).first()[0]
end_date

datetime.date(2021, 3, 31)

In [680]:
# отфильтруем изначальный датасет на основании start_date и end_date
df_6 = df.select('date', 'new_cases').where((F.col('date') >= start_date) & (F.col('date') <= end_date)\
& (F.col('location')=='Russia')).orderBy(F.col('date'))

In [681]:
df_6.show()

+----------+---------+
|      date|new_cases|
+----------+---------+
|2021-03-27|   8783.0|
|2021-03-28|   8979.0|
|2021-03-29|   8589.0|
|2021-03-30|   8162.0|
|2021-03-31|   8156.0|
+----------+---------+



In [682]:
# с помощью оконной функции и метода lag найдем количество случаев на день ранее
# и удалим строку с null которая нам и не нужна, таким образом мы получили данные по количеству новых случаев вчера
# и дельту для каждой даты нашей выборки согласно условию задачи
df_6.select('date', F.col('new_cases').alias('cases_today'), lag('new_cases', 1,)\
.over(W.orderBy('date')).alias('cases_yesterday'), (F.col('new_cases') - lag('new_cases', 1,).over(W.orderBy('date')))\
.alias('delta')).dropna().show()

+----------+-----------+---------------+------+
|      date|cases_today|cases_yesterday| delta|
+----------+-----------+---------------+------+
|2021-03-28|     8979.0|         8783.0| 196.0|
|2021-03-29|     8589.0|         8979.0|-390.0|
|2021-03-30|     8162.0|         8589.0|-427.0|
|2021-03-31|     8156.0|         8162.0|  -6.0|
+----------+-----------+---------------+------+



In [683]:
spark.stop()
sc.stop()